## 3-2 거리 변수 생성

In [15]:
from haversine import haversine
import pandas as pd
import numpy as np
import time
import math

In [2]:
data = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Seoul_location/종로구 실거래(13~18).xlsx")
school = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/서울 초중등학교 위치.xls")
subway = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/서울특별시 지하철.xlsx")
hospital = pd.read_csv("C:/Users/Yoo/Documents/Capstone/Factor/서울특별시 종합병원.csv", encoding = 'cp949')
market = pd.read_csv("C:/Users/Yoo/Documents/Capstone/Factor/서울특별시_전통시장(2020).csv", encoding = 'cp949')
library = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/서울특별시_공공도서관_현황(2019).xlsx")
factory = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/서울특별시_공해공장.xlsx")

## 3-2-1 학교 변수 생성

In [3]:
high_school = school[school['학교급구분'] == '고등학교'].reset_index(drop=True)
middle_school = school[school['학교급구분'] == '중학교'].reset_index(drop=True)
elementary_school = school[school['학교급구분'] == '초등학교'].reset_index(drop=True)

In [4]:
data['D_hsch'] = ''
data['D_msch'] = ''
data['D_esch'] = ''

In [5]:
# 고등학교 변수 생성
a = (data['lat'].values, data['lng'].values)
b = (high_school['위도'].values, high_school['경도'].values)

for i in range(len(data)):
    x = 0
    for j in range(len(high_school)):
        if(haversine((a[0][i],a[1][i]), (b[0][j],b[1][j]), unit = 'm') < 300):
            x = x + 1
    data.at[i,'D_hsch'] = x

In [6]:
#중학교 변수 생성
a = (data['lat'].values, data['lng'].values)
b = (middle_school['위도'].values, middle_school['경도'].values)

for i in range(len(data)):
    x = 0
    for j in range(len(middle_school)):
        if(haversine((a[0][i],a[1][i]), (b[0][j],b[1][j]), unit = 'm') < 300):
            x = x + 1
    data.at[i,'D_msch'] = x

In [7]:
#초등학교 변수 생성
a = (data['lat'].values, data['lng'].values)
b = (elementary_school['위도'].values, elementary_school['경도'].values)

for i in range(len(data)):
    x = 0
    for j in range(len(elementary_school)):
        if(haversine((a[0][i],a[1][i]), (b[0][j],b[1][j]), unit = 'm') < 300):
            x = x + 1
    data.at[i,'D_esch'] = x

## 3-2-2 지하철 변수 생성

In [8]:
data['D_sub'] = ''
data['D_trans'] = ''

In [9]:
# 지하철 변수 생성
a = (data['lat'].values, data['lng'].values)
b = (subway['위도'].values, subway['경도'].values)
c = subway['환승역'].values

for i in range(len(data)):
    x = 0
    y = 0
    for j in range(len(subway)):
        if(haversine((a[0][i],a[1][i]), (b[0][j],b[1][j]), unit = 'm') < 300):
            x = x + 1
            y += c[j]
    data.at[i,'D_sub'] = x
    data.at[i,'D_trans'] = y

## 3-2-3 병원 변수 생성

In [10]:
data['D_hos'] = ''

In [11]:
# 병원 변수 생성
a = (data['lat'].values, data['lng'].values)
b = (hospital['latitude'].values, hospital['longitude'].values)

for i in range(len(data)):
    x = 0
    for j in range(len(hospital)):
        if(haversine((a[0][i],a[1][i]), (b[0][j],b[1][j]), unit = 'm') < 300):
            x = x + 1
    data.at[i,'D_hos'] = x

## 3-2-4 전통시장 변수 생성

In [12]:
data.at[i,'D_market'] = ''

In [13]:
# 전통시장 변수 생성
a = (data['lat'].values, data['lng'].values)
b = (market['위도'].values, market['경도'].values)

for i in range(len(data)):
    x = 0
    for j in range(len(market)):
        if(haversine((a[0][i],a[1][i]), (b[0][j],b[1][j]), unit = 'm') < 300):
            x = x + 1
    data.at[i,'D_market'] = x

## 3-2-5 도서관 변수 생성

In [17]:
data['D_library'] = ''

In [16]:
# 결측값 0으로, 아닌 값 년도만
library['설립연도'] = library['설립연도'].apply(lambda b: 0 if math.isnan(b) == True else int(str(b)[0:4]))

In [18]:
# 도서관 변수 생성
a = (data['lat'].values, data['lng'].values, data['년'].values)
b = (library['위도'].values, library['경도'].values,library['설립연도'].values)

for i in range(len(data)):
    x = 0
    for j in range(len(library)):
        if(haversine((a[0][i],a[1][i]), (b[0][j],b[1][j]), unit = 'm') < 300 and a[2][i] > b[2][j]):
            x = x + 1
    data.at[i,'D_library'] = x

## 3-2-6 공장(공해) 변수 생성

In [ ]:
A = ['해당없음', '5종', '4종', '3종', '2종', '1종']
B = ['무', '유']
factory['sum'] = 0
data['D_factory'] = 0

In [ ]:
# 공장등록일 년도로 표현
factory['공장등록일'] = factory['공장등록일'].apply(lambda b: int(b[0:4]))

In [ ]:
# 점수 책정
for i, j in enumerate(A):
    factory.loc[factory['대기오염'] == j, 'sum'] = factory.loc[factory['대기오염'] == j, 'sum'] + i
    factory.loc[factory['수질오염'] == j, 'sum'] = factory.loc[factory['수질오염'] == j, 'sum'] + i
    
for i, j in enumerate(B):
    factory.loc[factory['소음/진동여부'] == j, 'sum'] = factory.loc[factory['소음/진동여부'] == j, 'sum'] + i

In [ ]:
# 공장(공해) 변수 생성
a = (data['lat'].values, data['lng'].values, data['년'].values)
b = (factory['fac_lat'].values, factory['fac_lng'].values,factory['공장등록일'].values)

for i in range(len(data)):
    for j in range(len(factory)):
        if(haversine((a[0][i],a[1][i]), (b[0][j],b[1][j]), unit = 'm') < 300 and a[2][i] > b[2][j]):
            data.at[i,'D_factory'] = data.at[i,'D_factory'] + factory.at[j, 'sum']

In [19]:
data.to_excel('종로구 실거래(13~18).xlsx', index = False)